# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
import shutil
import urllib.request

urllib.request.urlretrieve('https://s3.amazonaws.com/nist-srd/SD19/by_class.zip', 'data.zip')

from zipfile import ZipFile  
with ZipFile("data.zip", 'r') as zip: 
	zip.printdir() 
	zip.extractall()

In [ ]:
#shutil.move("/content/sample_data/mnist_test.csv", "/content/tt/")
#os.listdir("/content/images/72/hsf_3")
#shutil.rmtree("/content/training/")
#!ls /content/images/71
#shutil.rmtree("/content/training_small/")
#shutil.rmtree("/content/testing_small/")

In [ ]:
os.mkdir("images")
os.mkdir("training")
os.mkdir("testing")
for i in os.listdir("/content/by_class"):
  os.mkdir("training/"+i)
  os.mkdir("testing/"+i)
  os.mkdir("images/"+i)

In [ ]:
file_paths = []
for root, directories, files in os.walk("/content/by_class"):
    for filename in files:
        filepath = os.path.join(root, filename)
        file_paths.append(filepath)
        
#all images moved into images
for i in file_paths:
  i_split = i.split("/")
  class_name = i_split[3]
  ext_name = i_split[-1][-4:]
  file_name = i_split[-1]
  #print(f"i_split :{i_split}, classname: {class_name}, ext_name: {ext_name}, file_name = {filename}")
  if ext_name == ".png":
    shutil.move(i, "/content/images/"+class_name+"/")

In [ ]:
#moving files from images to training and testing
train_size_per = .8
for i in os.listdir("/content/images/"):
  #getting image files
  file_paths = []
  for root, directories, files in os.walk("/content/images/"+i):
      for filename in files:
          filepath = os.path.join(root, filename)
          file_paths.append(filepath)

  count = 0
  train_size = int(len(file_paths)*0.8)
  for j in file_paths:
      if count <= train_size:
        shutil.move(j, "/content/training/"+i+"/")
      else:
        shutil.move(j, "/content/testing/"+i+"/")
      count += 1

# Testing on small data size with 100 images of each for training and 20 for testing 

In [ ]:
#moving files from images to training and testing

os.mkdir("training_small")
os.mkdir("testing_small")
for i in os.listdir("/content/by_class"):
  os.mkdir("training_small/"+i)
  os.mkdir("testing_small/"+i)

for i in os.listdir("/content/training/"):  
  train_size_count = 0
  test_size_count = 0
  for root, directories, files in os.walk("/content/training/"+i):
      for filename in files:
          filepath = os.path.join(root, filename)
          if train_size_count <= 100:
            shutil.copy2(filepath, "/content/training_small/"+i+"/")
            train_size_count += 1
          elif test_size_count <= 20:
            shutil.copy2(filepath, "/content/testing_small/"+i+"/")
            test_size_count += 1
          else:
            break

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/training/',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 1236778 images belonging to 62 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/testing/',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 309145 images belonging to 62 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=2**11, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=2**8, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=62, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 100)

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dogs_1280p_0.jpg', target_size = (64, 64))
#test_image = image.load_img('/content/Section 40 - Convolutional Neural Networks (CNN)/dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [ ]:
print(prediction)